# IMPORT OF LIBRARIES

In [ ]:
!pip install torchserve torch-model-archiver torch-workflow-archiver

In [ ]:
!apt-get install -y libgl1-mesa-glx

In [ ]:
!apt-get install -y libglib2.0-0

In [ ]:
!apt-get install -y python3-distutils

In [ ]:
!pip install torch-model-archiver

In [ ]:
!pip install torchvision 

In [ ]:
!pip install opencv-python

In [ ]:
!pip install onnx

In [ ]:
!pip install coremltools

In [ ]:
!git clone https://github.com/IvanGarcia7/TORCHSERVER.git

In [ ]:
cd /content/TORCHSERVER

# CREATION OF A FOLDER TO STORE ALL THE FILES ON THE SERVER

In [ ]:
!mkdir '/content/model-server'

In [ ]:
!cp -r ./ressources/ '/content/model-server/ressources/'

In [ ]:
!chmod -R a+rw '/content/model-server/'

In [ ]:
cd '/content'

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
cd yolov5

In [ ]:
!pip install -r requirements.txt

# DOWNLOAD THE YOLO MODEL

In [ ]:
!/content/model-server/ressources/download_weights.sh

In [ ]:
!cp /content/yolov5/yolov5m.pt /content/model-server/ressources

In [ ]:
import torch
import torchvision

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
!python /content/yolov5/export.py --weights /content/yolov5/yolov5l.pt --img 640 --batch 1

In [ ]:
!torch-model-archiver --model-name my_model_name \
--version 0.1 --serialized-file /content/yolov5/yolov5l.torchscript \
--handler /content/model-server/ressources/torchserve_handler.py \
--extra-files /content/model-server/ressources/index_to_name.json,/content/model-server/ressources/torchserve_handler.py

In [ ]:
!mkdir /content/model-store/

In [ ]:
!mv my_model_name.mar /content/model-store

In [ ]:
cd /content/model-store

In [ ]:
!echo "inference_address=http://0.0.0.0:9080" >> config.properties

In [ ]:
!echo "management_address=http://0.0.0.0:9081" >> config.properties

In [ ]:
!echo "metrics_address=http://0.0.0.0:9082" >> config.properties

# START THE SERVER

In [ ]:
import os
os.system('torchserve --start \
           --ncs \
           --ts-config /content/model-store/config.properties \
           --model-store /content/model-store \
           --models my_model_name.mar')

# TEST

In [ ]:
!curl http://localhost:9080/ping

In [ ]:
cd /content/model-server

In [ ]:
!wget https://ichef.bbci.co.uk/news/768/cpsprodpb/7A5C/production/_113942313_mediaitem113942310.jpg

In [ ]:
!curl -X POST http://localhost:9080/predictions/my_model_name -T /content/model-server/_113942313_mediaitem113942310.jpg

# STOP THE SERVER

In [ ]:
os.system('torchserve --stop')